#### Spotlight

The Quickstart Example

In [ ]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

In [ ]:
dataset = get_movielens_dataset(variant='100K')

train, test = random_train_test_split(dataset)

In [ ]:
train

Not helpful... need to go deeper:

In [ ]:
train.user_ids

In [ ]:
train.item_ids

In [ ]:
train.ratings

And peek inside:

In [ ]:
train.tocsr().todense()

The model:

In [ ]:
model = ImplicitFactorizationModel(n_iter=3,
                                   loss='bpr')
model.fit(train)

mrr = mrr_score(model, test)

In [ ]:
mrr[:10]

### GitHub Stars

Data retrived from scraping GitHub:

In [ ]:
import pandas as pd 

df = pd.read_csv('data/stars.csv')

In [ ]:
df.sample(5)

In [ ]:
df['language'].value_counts()

In [ ]:
df = df[df.language == 'Python']
df = df[~df['repo'].isin(['maxhumber/gif', 'maxhumber/gazpacho'])]

In [ ]:
df.shape

In [ ]:
len(df['repo'].unique())

In [ ]:
len(df['user'].unique())

In [ ]:
df.head(3)

In [ ]:
from spotlight.interactions import Interactions

In [ ]:
# won't work
interactions = Interactions(df['user'], df['repo'])

"Everything must be a number"

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

In [ ]:
users = user_encoder.fit_transform(df['user'])
items = item_encoder.fit_transform(df['repo'])

In [ ]:
interactions = Interactions(users, items)

In [ ]:
interactions

Be a good Data Scientist:

In [ ]:
import numpy as np
from spotlight.cross_validation import random_train_test_split

train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=np.random.RandomState(42))

In [ ]:
model = ImplicitFactorizationModel(loss='pointwise', n_iter=20)

In [ ]:
model.fit(train)

In [ ]:
df.head(3)

Examining one user:

In [ ]:
garrry = df[df['user'] == 'garrrychan']

In [ ]:
garrry['repo'].values

Looking at the user_id:

In [ ]:
u = user_encoder.transform(['garrrychan'])

And all of the items:

In [ ]:
item_encoder.classes_

In [ ]:
preds = model.predict(u, np.arange(len(item_encoder.classes_)))

In [ ]:
pd.DataFrame({
    'repo': item_encoder.classes_,
    'pred': preds
}).sort_values('pred', ascending=False)

Evaulating the model:

In [ ]:
from spotlight.evaluation import precision_recall_score

In [ ]:
precision, recall = precision_recall_score(model, test, train, k=10)

In [ ]:
precision.mean()

In [ ]:
recall.mean()

Serialize:

In [ ]:
import torch 

torch.save(model, 'model.spot')

In [ ]:
del model

In [ ]:
model = torch.load('model.spot')

Predict on another random user:

In [ ]:
u = user_encoder.transform(['RandomOS'])

In [ ]:
model.predict(u, np.arange(len(item_encoder.classes_)))

In [ ]:
pd.DataFrame({
    'repo': item_encoder.classes_,
    'pred': model.predict(u, np.arange(len(item_encoder.classes_)))
}).sort_values('pred', ascending=False).head(20)

Actual likes:

In [ ]:
df[df['user'] == 'RandomOS']['repo']